#### Objectives

## Objective 1

- Project performs news title analysis on all stocks listed in the NYSE and Nasdaq. It then produces a list of the top 100 long options and top 100 short options
- Leave unweighted to take into account trending stocks
- do sentiment analysis for sectors too, so as to reccomend sector etfs
- add filter option for stocks such as sector and technical fundamentals, such as cap and pe
- list  stocks with best sentiment by sector


- Find pattern using previous news stories. use week 1 as base, and week as tuning. objective is to estimate week 2 ending value, or at least direction up/down

- add scales to the keywords
- translate to not waste headlines


# THERE ARE REQUEST LIMITS IN THE API. 50 MAX

In [1]:
#old code ignore this cell
import requests
import pandas as pd
import json

Stocklist=['XOM','NVDA','INTC','TSLA','F','T','KHC']
url = ('https://newsapi.org/v2/everything?'
       'q=XOM&'
       'from=2024-07-5&'
       'to=2024-06-5'
       'sortBy=popularity&'
       'apiKey=a7cd801675e34669b53c3e9f7cffc4a7')
# Use stock ticker not company name
#print(response.json())

response = requests.get(url)

# Need to store ticket and title
data=response.json()
#print(data.keys())


df=pd.DataFrame(data['articles'])
print(df.columns)
df=df.drop(columns=['source', 'author', 'description', 'url', 'urlToImage', 'publishedAt', 'content'])
if 'Ticker' not in df.columns:
    df.insert(0, 'Ticker', 'XOM')
#df['Ticker']='XOM'
df.to_csv('News.csv', index=False)

Index(['source', 'author', 'title', 'description', 'url', 'urlToImage',
       'publishedAt', 'content'],
      dtype='object')


In [4]:
import chardet 

# use the detect method to find the encoding
# 'rb' means read in the file as binary
with open('NYSELIST.csv', 'rb') as file:
    print(chardet.detect(file.read()))

{'encoding': 'Windows-1252', 'confidence': 0.7292082225034218, 'language': ''}


In [12]:
import requests
import pandas as pd
import json
import re
from datetime import datetime, timedelta
current_date = datetime.now().date()
date_30_days_ago = current_date - timedelta(days=30)


df_main=pd.DataFrame()
file_path = 'NYSELIST.csv'
Stocklist =  pd.read_csv("NYSELIST.csv", encoding="Windows-1252")
Stocklist.drop(columns=[' Company'])
#print(Stocklist[" Company"])
Stocklist=['Exxon','Nvidia','Intel','Tesla','Ford','Kraft Heinz']
#Stocklist[" Symbol"]
for i in Stocklist:
  #  print(i)
    url = ('https://newsapi.org/v2/everything?'
           f'q={i}&'
           f'from=q{current_date}&'
           f'to=q{date_30_days_ago}'
           'sortBy=popularity&'
           'apiKey=a7cd801675e34669b53c3e9f7cffc4a7')
    response = requests.get(url)
    data=response.json()
    if(data!=None):
        print(data)
        df=pd.DataFrame(data['articles'])
     #   df=df.drop(columns=['source', 'author', 'description', 'url', 'urlToImage', 'publishedAt', 'content'])
        df.insert(0, 'Ticker', i)   
        df_main = pd.concat([df_main, df])

    #df_main.append(df, ignore_index=True)

# Use stock ticker not company name
#print(response.json())
pattern = re.compile(r'[^\x00-\x7F]+')
def contains_non_ascii(row):
    return any(pattern.search(str(value)) for value in row)
df_main = df_main[~df_main.apply(contains_non_ascii, axis=1)]


#def contains_ticker(row):
#    return any(pattern.search(elem in for elem in Stocklist) for value in row)

mask = df_main['title'].apply(lambda text: any(element in text for element in Stocklist))
df_main=df_main[mask]

df_main.to_csv('News.csv', index=False)

# Need to store ticket and title

#print(data.keys())



#print(df.columns)


{'status': 'error', 'code': 'rateLimited', 'message': 'You have made too many requests recently. Developer accounts are limited to 100 requests over a 24 hour period (50 requests available every 12 hours). Please upgrade to a paid plan if you need more requests.'}


KeyError: 'articles'

In [32]:
(data['articles'][0])['title']

'Denali Advisors LLC Acquires Shares of 10,100 Exxon Mobil Co. (NYSE:XOM)'

## Keywords

### Positive

- Bought
- Acquired
- Grew
- Raised
- was up
- Climbed

### Negative

- Trimmed
- Sold
- Cut
- was down
- Crashed

In [18]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process


# Load the CSV file
file_path = 'News.csv'
df = pd.read_csv(file_path)
SentimentValue=0

# Define keywords for positive and negative sentiment

#original arrays
#positive_keywords = ['tops','jumps','purchase','increase','sign','giant','deal','bullish','growth', 'profit', 'success','safe','generate', 'win','won', 'gain', 'improve', 'raised', 'bought', 'buy', 'acquire','acquires', 'grew','gain','Rally', 'melt up', 'was up', 'up', 'climbed', 'Rally', 'Rallied']
#negative_keywords = ['boasts','cancel','suspension','stall','quashed','bearish','loss', 'lost','recalls', 'decline','disaster', 'problem', 'lawsuit', 'fail', 'drop', 'trimmed', 'sell', 'sold', 'cut', 'was down', 'crash', 'decrying', 'melt down', 'broken']


positive_keywords = ['tops','jumps','purchase','increase','sign','giant','deal','growth', 'profit', 'success','safe','generate', 'win','won', 'gain', 'improve', 'raised', 'bought', 'buy', 'acquire','acquires', 'grew','gain', 'was up', 'up', 'climbed']
very_positive_keywords=['bullish', 'rally', 'melt up', 'rallied']
negative_keywords = ['boasts','cancel','suspension','stall','quashed','loss', 'lost','recalls', 'decline','disaster', 'problem', 'lawsuit', 'fail', 'drop']
very_negative_keywords=['bearish', 'trimmed', 'sell', 'sold', 'cut', 'was down', 'crash', 'decrying', 'melt down', 'broken']
#the below doesnt work, first positive or negative word it sees, it marks , it ignores counters

def rule_based_labeling(title):
    title = title.lower()
    title=title.split(" ")
    count_positive=0
    count_negative=0
    for word in title:
        match, similarity=process.extractOne(word, positive_keywords)
        if(similarity>=80):
            count_positive+=1
        match, similarity=process.extractOne(word, negative_keywords)
        if(similarity>=80):
            count_negative+=1            
        match, similarity=process.extractOne(word, very_positive_keywords)
        if(similarity>=80):
            count_positive+=2
        match, similarity=process.extractOne(word, very_negative_keywords)
        if(similarity>=80):
            count_negative+=2
    return count_positive-count_negative
        


# Apply rule-based labeling
df['sentiment'] = df['title'].apply(rule_based_labeling)

output_file_path = 'news_with_sentiment.csv'
df.to_csv(output_file_path, index=False)


30
